In [1]:
import pandas as pd
import pickle
import tqdm
import sys
import os
sys.path.append("/aloy/home/acomajuncosa/Ersilia/ready-to-screen-enamine-real/src")
from src import download, smiles_to_ecfp, save_fingerprints, save_ids, upload

root = "."

# Get file
file = pickle.load(open(os.path.join("..", "files.pkl"), "rb"))[0]

# Download
sys.stderr.write("Downloading...\n")
# download(file, os.path.join("..", "tmp", file), "../service.json", "1P3DSLRpD0GL8th2B78b7OauEXp3I2ZY0")

Downloading...


15

In [2]:
# Get SMILES
sys.stderr.write("Getting SMILES...\n")
SMILES = pd.read_csv(os.path.join("..", "tmp", file)).values.tolist()

# # Calculate ECFP4s
# sys.stderr.write("Calculating ECFP4s...\n")
# OUTPUT_SMILES, X = smiles_to_ecfp(SMILES)

Getting SMILES...


In [3]:
from google.oauth2.service_account import Credentials
from googleapiclient.http import MediaFileUpload
from scipy.sparse import csr_matrix, save_npz
from rdkit.Chem import rdFingerprintGenerator
from googleapiclient.discovery import build
from rdkit import Chem
from tqdm import tqdm
import numpy as np
import pandas as pd
import httplib2
import os

In [11]:
def clip_sparse(vect, nBits=2048):
    """
    Convert a sparse RDKit fingerprint vector to a dense int8 list.

    Parameters
    ----------
    vect : rdkit.DataStructs.cDataStructs.ExplicitBitVect or similar
        Sparse fingerprint vector with nonzero elements.
    nBits : int, optional, default=2048
        Length of the dense output vector.

    Returns
    -------
    list of int
        Dense list representation of the fingerprint, where values are
        clipped to the maximum representable int8 value.
    """
    l = [0] * nBits
    for i, v in vect.GetNonzeroElements().items():
        l[i] = v if v < np.iinfo(np.int8).max else np.iinfo(np.int8).max
    return np.array(l, dtype=np.int8)

In [12]:
mfpgen = rdFingerprintGenerator.GetMorganGenerator(radius=3, fpSize=2048)
OUTPUT_SMILES, X = [], []
for smi, _id in tqdm(SMILES):
    mol = Chem.MolFromSmiles(smi)
    ecfp = mfpgen.GetCountFingerprint(mol)
    ecfp = clip_sparse(ecfp)
    X.append(ecfp)
    # OUTPUT_SMILES.append([smi, _id])

  2%|▏         | 219994/10000000 [00:27<20:36, 7911.95it/s]


KeyboardInterrupt: 